In [9]:
from pandas_datareader import data
import datetime
from bokeh.plotting import figure, show, output_file

## 1. Importing the Dataset
In this project, we will make use of the stock price of Apple, with the ticker name 'AAPL' for the period of 10 trading days, starting from May 16, 2022 up to May 28, 2022. The dataset is retrieved from Yahoo! Finance. The dataset contains the opening, closing, highest, and lowest stock price per each trading days as well as the amount/volume of stocks traded per day. 

In [25]:
start=datetime.datetime(2022,5,16)
end=datetime.datetime(2022,5,28)
df = data.DataReader(name= "AAPL", data_source="yahoo", start=start, end=end)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-05-16,147.520004,144.179993,145.550003,145.539993,86643800,145.539993
2022-05-17,149.770004,146.679993,148.860001,149.240005,78336300,149.240005
2022-05-18,147.360001,139.899994,146.850006,140.820007,109742900,140.820007
2022-05-19,141.660004,136.600006,139.880005,137.350006,136095600,137.350006
2022-05-20,140.699997,132.610001,139.089996,137.589996,137194600,137.589996


## 2. Building Chart Candlesticks with Bokeh Rectangles
To build the candlestick, we will make use of Bokeh, a Python library utilized to create interactive visualizations for modern web browsers. In addition, Bokeh also enables the creation of JavaScript-powered visualizations without writing any JavaScript yourself. In this case, we will apply a rectangular class in Bokeh to create each candle. Before plotting, we need to preprocess our data to provide all necessary arguments for Bokeh. Those arguments include :

1. The x-coordinates of the centers of the rectangles
It will be every index of the trading day when the stock price is increasing or decreasing. A new column called `Status` has been added to the Dataframe to show movement of stock price.
2. The y-coordinates of the centers of the rectangles.
It will be the middle point between the open and stock price. A new column called 'Middle' has been calculated. 
3. The overall widths of the rectangles
It will be the duration of trading in milliseconds. 
4. The overall heights of the rectangles.
It will be taken from absolute value of the result of substraction between opening and closing price. 

In [26]:
def increase_decrease(close_price, open_price):
    """
    This function specify whether the stock price is increasing or decreasing for each trading day
    """
    if close_price > open_price:
        value = "Increase"
    elif close_price < open_price:
        value = "Decrease"
    else:
        value="Equal"
    return value

df["Status"]=[increase_decrease(close_price, open_price) for close_price, open_price in zip(df.Close, df.Open)]

In [27]:
df["Middle"]= (df.Open+df.Close)/2
df["Width"] = 12*60*60*1000
df["Height"]= abs(df.Open-df.Close)

In [28]:
df.head()

,High,Low,Open,Close,Volume,Adj Close,Status,Middle,Width,Height
Date,,,,,,,,,,
2022-05-16,147.520004,144.179993,145.550003,145.539993,86643800,145.539993,Decrease,145.544998,43200000,0.010010
2022-05-17,149.770004,146.679993,148.860001,149.240005,78336300,149.240005,Increase,149.050003,43200000,0.380005
2022-05-18,147.360001,139.899994,146.850006,140.820007,109742900,140.820007,Decrease,143.835007,43200000,6.029999
2022-05-19,141.660004,136.600006,139.880005,137.350006,136095600,137.350006,Decrease,138.615005,43200000,2.529999
2022-05-20,140.699997,132.610001,139.089996,137.589996,137194600,137.589996,Decrease,138.339996,43200000,1.500000


In [24]:
p = figure(x_axis_type='datetime', width=1000, height = 300)
p.title.text = "Candlestick Chart"



p.rect(df.index[df.Status=="Increase"], df.Middle[df.Status=="Increase"], 
       df.Width, df.Height[df.Status=="Increase"], fill_color="green", line_color="black") 

p.rect(df.index[df.Status=="Decrease"], df.Middle[df.Status=="Decrease"], 
      df.Width, df.Height[df.Status=="Decrease"], fill_color="red", line_color="black") 

output_file("CS.html")
show(p)